In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [2]:
%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



#from sklearn.ensemble import BaggingClassifier, BaggingRegressor

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common




import time
import pandas as pd


In [3]:
import ray
from ray import tune
from ray.data import Dataset
# from ray.data.preprocessors import LabelEncoder, SimpleImputer, StandardScaler
from ray.data import preprocessors # import LabelEncoder, SimpleImputer, StandardScaler
from lightgbm_ray import RayDMatrix, RayParams, train



In [5]:
rayer.get_global_cluster(num_cpus=6, ip='1')

2022-11-29 01:36:09,785	INFO packaging.py:499 -- Creating a file package for local directory '/home/wasif/python-asd/xai/'.
2022-11-29 01:36:09,807	WARNING packaging.py:386 -- File /home/wasif/python-asd/xai/train.csv is very large (134.00MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/wasif/python-asd/xai/train.csv']})`
2022-11-29 01:36:12,470	INFO packaging.py:343 -- Pushing file package 'gcs://_ray_pkg_a35246b6b3039630.zip' (196.34MiB) to Ray cluster...
2022-11-29 01:36:14,954	INFO packaging.py:352 -- Successfully pushed file package 'gcs://_ray_pkg_a35246b6b3039630.zip'.


Client Ray installation incompatible with server: client is 2022-07-24, server is 2022-10-05


In [12]:
ray.__version__

'2.0.1'

In [32]:
# parse_options = csv.ParseOptions(delimiter=";")
# ds_train = ray.data.read_csv('/home/wasif/python-asd/xai/train.csv', parse_options=parse_options, partition_filter=None)
# ds_train = ds_train.repartition(5)

In [6]:
ds_train = pd.read_csv('/home/wasif/python-asd/xai/train.csv', sep = ',')
df_X = ds_train.loc[:, ~ds_train.columns.isin(['target', 'ID_code'])]
df_y = ds_train['target'].astype(str)
# ds_train.head()

In [ ]:
df_X.values

In [37]:
ray.data.from_items(df_X.values)

Dataset(num_blocks=201, num_rows=93047, schema={__value__: ArrowTensorType(shape=(200,), dtype=double)})

In [14]:
ray.data.from_items(df_y)

Dataset(num_blocks=201, num_rows=93047, schema=<class 'str'>)

In [6]:
def get_ray_kaggle_trans_ds():
    ds_train = pd.read_csv('/home/wasif/python-asd/xai/train.csv', sep = ',')
    df_X = ds_train.loc[:, ~ds_train.columns.isin(['target', 'ID_code'])]
    df_y = ds_train['target'].astype(str)
    ds_x = ray.data.from_pandas(df_X.values)
    ds_y = ray.data.from_items(df_y)

    list_label_cols = None

    for col in df_X:
        if df_X[col].dtype == object:
            print(col)
            list_label_cols.append(col)

    ### do label encoding        
    if not list_label_cols is None:
        encoder = preprocessors.LabelEncoder(label_column=list_label_cols)
        ds_x = encoder.fit_transform(ds_x)

    ### fill in NaN
    preprocessor = preprocessors.SimpleImputer(columns=df_X.columns, strategy="constant", fill_value=-1)
    ds_x = preprocessor.fit_transform(ds_x)

    ### Scale to standard
    ds_x = preprocessors.StandardScaler(columns=df_X.columns).fit_transform(ds_x)
    return ds_x, ds_y

In [ ]:
### pandas to ray.dataset
ds_train = pd.read_csv('/home/wasif/python-asd/xai/train.csv', sep = ',')
df_X = ds_train.loc[:, ~ds_train.columns.isin(['target', 'ID_code'])]
df_y = ds_train['target']
ds_x = ray.data.from_pandas(df_X)
ds_y = ray.data.from_pandas(df_y)

list_label_cols = None

for col in df_X:
    if df_X[col].dtype == object:
        print(col)
        list_label_cols.append(col)

### do label encoding        
if not list_label_cols is None:
    encoder = LabelEncoder(label_column=list_label_cols)
    ds_x = encoder.fit_transform(ds_x)

### fill in NaN
preprocessor = SimpleImputer(columns=df_X.columns, strategy="constant", fill_value=-1)
ds_x = preprocessor.fit_transform(ds_x)

### Scale to standard
ds_x = StandardScaler(columns=df_X.columns).fit_transform(ds_x)

return ds_x, ds_y

In [53]:
df_X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [58]:
list_label_cols = None

for col in df_X:
    if df_X[col].dtype == object:
        print(col)
        list_label_cols.append(col)

### do label encoding        
if not list_label_cols is None:
    encoder = LabelEncoder(label_column=list_label_cols)
    ds = encoder.fit_transform(ds)

### fill in NaN
preprocessor = SimpleImputer(columns=df_X.columns, strategy="constant", fill_value=-1)
ds = preprocessor.fit_transform(ds)

### Scale to standard
ds = StandardScaler(columns=df_X.columns).fit_transform(ds)


Map_Batches: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


In [ ]:
encoder = LabelEncoder(label_column=df_X.columns)

In [43]:
encoder = LabelEncoder(label_column=df_X.columns)
#ds = encoder.fit_transform(ds)

In [34]:
#ds
preprocessor = SimpleImputer(columns=ds_train.columns, strategy="constant", fill_value=-1)
ds = preprocessor.fit_transform(ds)



Map_Batches: 100%|██████████| 1/1 [00:02<00:00,  2.30s/it]


,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93042,train_93042,0,5.7154,7.5998,10.9337,9.2759,10.3940,-4.6492,4.8223,16.9555,...,1.0075,10.5036,3.2955,7.1503,17.2527,-0.3051,2.2054,8.5501,18.4327,7.3441
93043,train_93043,0,15.6705,-5.6945,10.3854,7.0876,10.9459,-10.2267,6.4689,16.5037,...,-4.4809,9.6724,2.6984,8.1799,19.8712,0.6613,0.3818,9.2073,17.5551,-2.1798
93044,train_93044,0,12.9178,2.5186,10.7434,6.5086,11.7769,0.6535,4.8543,14.7308,...,2.4613,3.9535,1.4656,5.3360,19.8530,-0.3346,1.8112,8.4936,17.1528,-5.8872
93045,train_93045,0,8.6149,1.7525,12.5942,3.3936,11.6596,5.7688,4.6281,10.0646,...,9.8929,7.5062,2.5473,4.1728,13.9554,0.6746,-4.0613,9.3766,17.0889,-1.8687


In [36]:
StandardScaler().fit_transform(ds)

TypeError: __init__() missing 1 required positional argument: 'columns'

In [22]:
def get_ray_kaggle_housing_data():
    ds_train = pd.read_csv('/home/wasif/python-asd/xai/train.csv', sep = ',')
    # ds_test = pd.read_csv('/home/wasif/python-asd/xai/test.csv', sep = ',')
    
    # ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    #ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    #ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target'].astype(str)
    
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)# , random_state=config.rand_state)

    ss = StandardScaler()
    X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
    X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)    
    
    return ray.data.from_pandas(X_train_scalar), ray.data.from_pandas(df_y)
    

In [7]:
def load_kaggle_housing_data():
    ds_train = pd.read_csv('/home/wasif/python-asd/xai/train.csv', sep = ',')
    #ds_test = pd.read_csv('/home/wasif/python-asd/xai/test.csv', sep = ',')
    # parse_options = csv.ParseOptions(delimiter=";")
    
    # ds_train = ray.data.read_csv('/home/wasif/python-asd/xai/train.csv', parse_options=parse_options, partition_filter=None)
    #ds_test = ray.data.read_csv('/home/wasif/python-asd/xai/test.csv', parse_options=parse_options, partition_filter=None)
    
    # ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    # ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    # ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target']

    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)# , random_state=config.rand_state)


    ss = StandardScaler()
    X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
    X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)
    return X_train_scalar, df_y
    #return ray.put(X_train_scalar),  ray.put(y_train)




In [46]:
train_x, train_y = load_kaggle_housing_data()
# get_ray_kaggle_housing_data()

In [47]:
train_set = RayDMatrix(train_x, label=train_y)

In [51]:
train_set.get_data(num_actors=2, rank=0)

{'data':         ID_code     var_0     var_1     var_2     var_3     var_4     var_5  \
 0     -1.386561 -0.995761  1.622846 -1.364274 -0.478700 -0.065613  0.763613   
 2      1.119155 -0.054380 -1.294378  1.519635 -1.470481 -0.138481 -0.770973   
 4      0.158913  0.824047 -0.454732 -0.184429 -0.186739  0.741197 -0.774328   
 6      1.243304  0.626052 -0.505084 -0.166830 -0.587235  1.569039  0.144429   
 8      0.513789  0.935661 -1.248637  2.013369  0.325413  0.621070  1.288814   
 ...         ...       ...       ...       ...       ...       ...       ...   
 62332 -1.715471  0.413041  1.287885  0.374951 -1.139562 -1.169712 -0.786399   
 62334 -0.509605 -1.298339  1.674777 -0.037590  0.697775  1.018691  0.492937   
 62336  1.019004  0.886836 -1.640855 -0.762701  0.965016 -0.330797  0.072226   
 62338  0.792883 -0.868572 -1.265454  0.848957 -0.253293  0.180457  1.077163   
 62340  0.207577 -0.246921 -0.483631  0.307290 -0.958378  0.573563 -1.117308   
 
           var_6     var_7    

In [20]:
train_x, train_y = get_ray_kaggle_trans_ds()

# get_data_source

Map_Batches: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


In [22]:
train_set = RayDMatrix()

In [25]:
train_set.load_data()

RuntimeError: Trying to shard data for 2 actors, but the maximum number of shards (i.e. the number of data rows) is 1. Consider using fewer actors.

In [ ]:
X_train_ref = ray.put(X_train_scalar)
y_train_ref = ray.put(y_train)
X_test_ref = ray.put(X_test_scalar)
y_test_ref = ray.put(y_test)

In [8]:
def train_model(config):
    #train_x, train_y = load_breast_cancer(return_X_y=True)
    train_x, train_y = load_kaggle_housing_data()
    # train_x, train_y = get_ray_kaggle_trans_ds()
    train_set = RayDMatrix(train_x, label=train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        #evals_result=evals_result,
        #valid_sets=[train_set],
        #valid_names=["train"],
        verbose_eval=False,
        ray_params=ray_params)
    
    # bst.booster_.save_model("model.lgbm")

In [9]:

num_actors = 2
num_cpus_per_actor = 2

ray_params = RayParams(num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)


# Specify the hyperparameter search space.
config = {
    # "objective": 'poisson',
    "objective": "binary",
    "metric": ["binary_logloss"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

#func = lambda trial: train_model(trial, arg1, arg2)


In [10]:

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    #metric="train-binary_error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())

print("Best hyperparameters", analysis.best_config)

2022-11-29 01:38:11,512	INFO tune.py:883 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2022-11-29 01:38:12,721	ERROR services.py:1365 -- Failed to start the dashboard: Ray Client is not connected. Please connect by calling `ray.init`.
2022-11-29 01:38:12,722	ERROR services.py:1366 -- Ray Client is not connected. Please connect by calling `ray.init`.
Traceback (most recent call last):
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/_private/services.py", line 1312, in start_dashboard
    dashboard_url = ray.experimental.internal_kv._internal_kv_get(
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return getattr(ray, func.__name__)(

AssertionError: 